In [14]:
%pylab inline
from snakebite.client import Client
import pyaml as yaml
import sys
import pythonrun
reload(pythonrun)
import os
import seaborn as sns
from sklearn import metrics
import scala_python_endive_wrapper
reload(scala_python_endive_wrapper)
from scala_python_endive_wrapper import *
import pandas as pd
import os
from sklearn.cluster import KMeans

Populating the interactive namespace from numpy and matplotlib


/home/eecs/akmorrow/anaconda2/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['power', 'gamma', 'random', 'fft', 'linalg', 'info']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [10]:
hdfsclient = Client("amp-bdg.master.amplab.net", 8020, use_trash=False)
notebook_stdout = sys.stdout
notebook_stderr = sys.stderr
terminal_stdout = open('/dev/stdout', 'w')
terminal_stderr = open('/dev/stderr', 'w')
logpath = "/tmp/log"

In [11]:
executor_mem = "200g"
cores_per_executor = 16
num_executors = 8
EGR1_PATH = 'hdfs://amp-spark-master.amp:8020/user/vaishaal/endive-data/EGR1'

In [12]:
# this is where the magic happens
results = [] 
num_filters = 256
gammas = [0.1]
dnase_gammas = [0.1]
kmer_size = 8
dnase_kmer_sizes = [8]

In [ ]:
chromosomes = map(lambda x: "chr{0}".format(x+1), range(23))
# CHANGE ME BASED ON ENUM + TF
egr1_cell_types = string_to_enum_celltypes(['GM12878', 'H1hESC', 'HCT116', 'MCF7'])
results_dfs = []
for gamma in gammas:
    for dnase_gamma in dnase_gammas:
        for dnase_kmer_size in dnase_kmer_sizes:
            feat_out_loc = "/user/akmorrow/featurizedWindows/sequence/small/{0}-filters/{1}-gamma/{3}-kmers/replicate-0"\
              .format(num_filters, gamma, dnase_gamma, kmer_size, dnase_kmer_size)
            res = run_kitchensink_featurize_pipeline(EGR1_PATH,
                           "/tmp/filters.csv", 
                           logpath, 
                           seed=0,
                           cores_per_executor=cores_per_executor,
                           num_filters=num_filters,
                           sample=0.001,
                           alphabet_size=4,
                           kmer_size=kmer_size,
                           gamma=gamma,
                           executor_mem=executor_mem,
                           num_partitions=cores_per_executor*num_executors,
                           featuresOutput=feat_out_loc,
                           num_executors=num_executors)
#             meta = {}
#             meta['dnase_kmer_size'] = dnase_kmer_size
#             meta['dnase_gamma'] = dnase_gamma
#             meta['gamma'] = gamma
#             results_df = cross_validate(feat_out_loc, 
#                             hdfsclient, 
#                             chromosomes, 
#                             egr1_cell_types,
#                             numHoldOutChr=1, 
#                             numHoldOutCell=1,
#                             cores_per_executor = 32,
#                             num_executors = 14,
#                             executor_mem="150g",
#                             num_folds=1, 
#                             negativeSamplingFreqs=[0.1],
#                             regs=[1e-8],
#                             other_meta=meta)
#             print(results_df)
#             results_dfs.append(results_df)
            
            


    


{'kmerLength': 8, 'reference': '/home/eecs/akmorrow/ADAM/endive/workfiles/hg19.2bit', 'numPartitions': 2000, 'dnaseBams': '/data/anv/DREAMDATA/dnase_bams/merged_coverage/', 'saveTestPredictions': 'predictions/', 'aggregatedSequenceOutput': 'hdfs://amp-spark-master.amp:8020/user/vaishaal/endive-data/EGR1', 'featuresOutput': '/user/akmorrow/featurizedWindows/sequence/small/256-filters/0.1-gamma/8-kmers/replicate-0', 'seed': 0, 'dnaseNarrow': '/data/anv/DREAMDATA/DNASE/peaks/relaxed/', 'approxDim': 256, 'featurizeSample': 0.001, 'alphabetSize': 4, 'readFiltersFromDisk': True}


In [75]:
pd.concat(results_dfs)

,dnase_gamma,dnase_kmer_size,gamma,negativeSamplingFreq,reg,test_celltypes,test_chromosomes,train_auPRC,train_auROC,train_recall_at_05_fdr,train_recall_at_10_fdr,train_recall_at_25_fdr,train_recall_at_50_fdr,val_auPRC,val_auROC,val_recall_at_05_fdr,val_recall_at_10_fdr,val_recall_at_25_fdr,val_recall_at_50_fdr
0,0.1,8,0.1,1,1.000000e-08,[3],[chr12],0.286507,0.940043,0.081238,0.104449,0.145390,0.221470,0.056017,0.900088,0.035714,0.035714,0.035714,0.035714
1,0.1,8,0.1,1,1.000000e-01,[3],[chr12],0.283020,0.941194,0.076725,0.101547,0.142166,0.215023,0.056503,0.899537,0.035714,0.035714,0.035714,0.035714


In [ ]:
print(feat_out_loc)
results_df = cross_validate(feat_out_loc, 
                            hdfsclient, 
                            chromosomes, 
                            egr1_cell_types,
                            numHoldOutChr=1, 
                            numHoldOutCell=1,
                            cores_per_executor = 32,
                            num_executors = 14,
                            executor_mem="150g",
                            num_folds=1, 
                            negativeSamplingFreqs=[0.001],
                            regs=[1e-8])
    

/user/vaishaal/featurizedWindows/sequence/small/256-filters/0.1-gamma/0.1-dnase-gamma/8-kmers/8-dnase-kmers/replicate-0
FOLD 0
HOLDING OUT CHROMOSOMES ['chr12']
HOLDING OUT CELL TYPES [3]
RUNING SOLVER WITH REG=1e-08


In [ ]:
results_df


0.88036847526310447

In [35]:
feat_out_loc

'/user/vaishaal/featurizedWindows/sequence/tiny/256-filters/1.0-gamma/1.0-dnase-gamma/8-kmers/100-dnase-kmers/replicate-0'

In [61]:
old_result_dfs = pd.concat(results_dfs)